In [1]:
import cstrees.cstree as ct
import numpy as np
import networkx as nx
import itertools
# CStree from Figure 1 in (Duarte & Solus, 2022)
np.random.seed(1)

In [6]:

p=4
cards = [2] * p
co = ct.CausalOrder(range(p))
tree = ct.CStree(co)


#stage = ct.sample_random_stage(cards,2)
#stage.set_random_params(cards)

tree.set_cardinalities(cards)

# These do not have to be in a dict like this as the levels are
# determined from the length of the tuples.

tree.set_stages({
    0: [],
    1: [ct.Stage([{0, 1}, 0])],    # Green
    2: [ct.Stage([0, {0, 1}, 0]),  # Blue
        ct.Stage([0, {0, 1}, 1]),  # Orange
        ct.Stage([1, {0, 1}, 0])]  # Red
})


tree.set_random_stage_parameters()

a = tree.plot()
a.draw("testplot.png")

x = tree.sample(5)
# print(x)
rels = tree.csi_relations()

print("Initial rels")
print(rels)
adjmats = tree.to_minimal_context_graphs()

i = 1
for key, graph in adjmats.items():
    agraph = nx.nx_agraph.to_agraph(graph)
    agraph.layout("dot")

    #agraph.draw(str(key) + "_csi.png", args="-Glabel=Context:"+str(key) +"   ")
    agraph.draw(str(key) + "_csi.png", args='-Glabel="'+str(key)+'"   ')
    # print(graph.nodes)
    # print(graph.edges())
    i += 1


[((), (1,)), ((), (0,)), ((1,), (1, 1)), ((1,), (1, 0)), ((1, 1), (1, 1, 1)), ((1, 1), (1, 1, 0)), ((1, 1, 1), (1, 1, 1, 1)), ((1, 1, 1), (1, 1, 1, 0)), ((1, 1, 0), (1, 1, 0, 1)), ((1, 1, 0), (1, 1, 0, 0)), ((1, 0), (1, 0, 1)), ((1, 0), (1, 0, 0)), ((1, 0, 1), (1, 0, 1, 1)), ((1, 0, 1), (1, 0, 1, 0)), ((1, 0, 0), (1, 0, 0, 1)), ((1, 0, 0), (1, 0, 0, 0)), ((0,), (0, 1)), ((0,), (0, 0)), ((0, 1), (0, 1, 1)), ((0, 1), (0, 1, 0)), ((0, 1, 1), (0, 1, 1, 1)), ((0, 1, 1), (0, 1, 1, 0)), ((0, 1, 0), (0, 1, 0, 1)), ((0, 1, 0), (0, 1, 0, 0)), ((0, 0), (0, 0, 1)), ((0, 0), (0, 0, 0)), ((0, 0, 1), (0, 0, 1, 1)), ((0, 0, 1), (0, 0, 1, 0)), ((0, 0, 0), (0, 0, 0, 1)), ((0, 0, 0), (0, 0, 0, 0))]
[(1,), (0,)]
[0.02440535 0.97559465]
[(1, 1), (1, 0)]
[0.39768183 0.60231817]
[(1, 1, 1), (1, 1, 0)]
[0.4065972 0.5934028]
[(1, 1, 1, 1), (1, 1, 1, 0)]
[0.31859409 0.68140591]
[(1, 1, 0, 1), (1, 1, 0, 0)]
[0.24186559 0.75813441]
[(1, 0, 1), (1, 0, 0)]
[0.48163431 0.51836569]
[(1, 0, 1, 1), (1, 0, 1, 0)]
[0.690

KeyError: 'cond_prob'